In [2]:
!pip install ta-lib

   ---------------------------------------- 0.0/883.4 kB ? eta -:--:--
    --------------------------------------- 20.5/883.4 kB ? eta -:--:--
   -- ------------------------------------ 61.4/883.4 kB 825.8 kB/s eta 0:00:01
   ---- --------------------------------- 102.4/883.4 kB 845.5 kB/s eta 0:00:01
   ------ --------------------------------- 153.6/883.4 kB 1.0 MB/s eta 0:00:01
   --------- ---------------------------- 215.0/883.4 kB 939.4 kB/s eta 0:00:01
   ------------ --------------------------- 286.7/883.4 kB 1.0 MB/s eta 0:00:01
   ---------------- ----------------------- 368.6/883.4 kB 1.1 MB/s eta 0:00:01
   ------------------------ --------------- 542.7/883.4 kB 1.6 MB/s eta 0:00:01
   ------------------------ --------------- 542.7/883.4 kB 1.6 MB/s eta 0:00:01
   ------------------------ --------------- 542.7/883.4 kB 1.6 MB/s eta 0:00:01
   ------------------------------------- -- 819.2/883.4 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------  880.6/883.4 k

DEPRECATION: Loading egg at f:\anaconda\lib\site-packages\ibapi-10.33.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
import pandas as pd
data = pd.read_csv('all_stocks_ohlc.csv')
data.head()  # Display the first few rows of the DataFrame

,timestamp,open,high,low,close,volume,Open Interest,symbol
0,2025-07-25T15:15:00+05:30,2552.6,2552.8,2545.0,2549.5,47654,0,ADANIENT
1,2025-07-25T15:00:00+05:30,2549.7,2552.9,2544.1,2552.7,31219,0,ADANIENT
2,2025-07-25T14:45:00+05:30,2550.0,2551.5,2546.0,2549.7,12152,0,ADANIENT
3,2025-07-25T14:30:00+05:30,2550.6,2553.0,2547.2,2550.0,5457,0,ADANIENT
4,2025-07-25T14:15:00+05:30,2549.2,2553.8,2546.7,2550.8,11238,0,ADANIENT


In [4]:
data ['timestamp'] = pd.to_datetime(data['timestamp'])
data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d  %H:%M:%S')
data.head()  # Display the first few rows of the DataFrame

,timestamp,open,high,low,close,volume,Open Interest,symbol
0,2025-07-25 15:15:00,2552.6,2552.8,2545.0,2549.5,47654,0,ADANIENT
1,2025-07-25 15:00:00,2549.7,2552.9,2544.1,2552.7,31219,0,ADANIENT
2,2025-07-25 14:45:00,2550.0,2551.5,2546.0,2549.7,12152,0,ADANIENT
3,2025-07-25 14:30:00,2550.6,2553.0,2547.2,2550.0,5457,0,ADANIENT
4,2025-07-25 14:15:00,2549.2,2553.8,2546.7,2550.8,11238,0,ADANIENT


In [5]:
# Filter only the daily closing candles at 15:15:00
# Adjust time depending on your exact last candle time in your data
closing_time = pd.to_datetime("15:15:00").time()
daily_close_df = data[data['timestamp'] == closing_time].copy()

# Sort by symbol and date
daily_close_df = daily_close_df.sort_values(['symbol', 'timestamp']).reset_index(drop=True)

results = []
symbols = daily_close_df['symbol'].unique()

In [ ]:
import pandas as pd
import numpy as np
import talib
from datetime import timedelta

# Load data
df = pd.read_csv("all_stocks_ohlc.csv", parse_dates=['timestamp'])

# Ensure columns are lowercase
df.columns = [col.lower() for col in df.columns]

# Sort for safety
df.sort_values(by=['timestamp', 'symbol'], inplace=True)

# --- STEP 1: Add RSI and aggregate data ---
def add_rsi(data):
    data['rsi'] = talib.RSI(data['close'], timeperiod=14)
    return data

df = df.groupby('symbol', group_keys=False).apply(add_rsi).reset_index(drop=True)

# Convert timestamp to datetime and extract date
df['date'] = df['timestamp'].dt.date
df['time'] = df['timestamp'].dt.time

# Daily aggregations
daily = df.groupby(['symbol', 'date']).agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum',
}).reset_index()

# Weekly high
weekly_high = daily.groupby('symbol').rolling(window=5, on='date')['high'].max().reset_index()
weekly_high.rename(columns={'high': 'weekly_high'}, inplace=True)
daily = daily.merge(weekly_high[['symbol', 'date', 'weekly_high']], on=['symbol', 'date'], how='left')

# Previous day high
daily['prev_day_high'] = daily.groupby('symbol')['high'].shift(1)

# 5-day average volume
daily['avg_5_vol'] = daily.groupby('symbol')['volume'].rolling(window=5).mean().reset_index(level=0, drop=True)

# --- STEP 2: Generate Buy Signals (EOD) ---
daily['signal'] = (
    (daily['volume'] > 1.5 * daily['avg_5_vol']) &
    (daily['close'] > daily['prev_day_high']) &
    (daily['close'] > daily['weekly_high'])
)

# For RSI, use 5-min df on daily EOD
eod_rsi = df.groupby(['symbol', 'date'], group_keys=False).apply(
    lambda x: (x['rsi'].iloc[-1] > 70) and (x['rsi'] > 70).any()
).reset_index().rename(columns={0: 'rsi_signal'})


daily = daily.merge(eod_rsi, on=['symbol', 'date'], how='left')
daily['final_signal'] = daily['signal'] & daily['rsi_signal']

# --- STEP 3: Execute Next Day Trade ---
results = []

for idx, row in daily.iterrows():
    if not row['final_signal']:
        continue

    symbol = row['symbol']
    signal_date = pd.to_datetime(row['date'])
    next_day = signal_date + timedelta(days=1)

    # Get next day's data
    next_day_data = df[(df['symbol'] == symbol) & (df['timestamp'].dt.date == next_day.date())]
    
    # Get 9:15 to 9:30 candle
    candle = next_day_data[
        (next_day_data['timestamp'].dt.time >= pd.to_datetime("09:15").time()) &
        (next_day_data['timestamp'].dt.time <= pd.to_datetime("09:30").time())
    ]

    if candle.empty:
        continue

    entry = candle['high'].max() + 0.05  # buffer
    sl = candle['low'].min()
    target = entry + 1.5 * (entry - sl)

    # Track trade intraday
    day_data = next_day_data[next_day_data['timestamp'].dt.time >= pd.to_datetime("09:30").time()]
    entry_hit = day_data[day_data['high'] >= entry]

    if entry_hit.empty:
        continue  # trade not triggered

    entry_time = entry_hit.iloc[0]['timestamp']
    trade_data = day_data[day_data['timestamp'] >= entry_time]

    # SL/Target check
    hit_price = None
    exit_time = None
    result = "Time Exit"
    for _, r in trade_data.iterrows():
        if r['low'] <= sl:
            hit_price = sl
            exit_time = r['timestamp']
            result = "SL"
            break
        elif r['high'] >= target:
            hit_price = target
            exit_time = r['timestamp']
            result = "Target"
            break

    if not hit_price:
        # Time exit at 3:20
        exit_row = trade_data[trade_data['timestamp'].dt.time <= pd.to_datetime("15:20").time()].iloc[-1]
        hit_price = exit_row['close']
        exit_time = exit_row['timestamp']

    pnl = hit_price - entry

    results.append({
        'symbol': symbol,
        'signal_date': signal_date,
        'entry_date': next_day,
        'entry_time': entry_time,
        'entry_price': entry,
        'exit_price': hit_price,
        'exit_time': exit_time,
        'result': result,
        'pnl': pnl
    })

print(f"Number of trades: {len(results)}")
if results:
    print("Keys in first trade dict:", results[0].keys())
else:
    print("⚠ No trades generated — conditions likely too strict.")


# --- STEP 4: Backtest Summary ---
results_df = pd.DataFrame(results)

if not results_df.empty and 'pnl' in results_df.columns:
    results_df['cumulative_pnl'] = results_df['pnl'].cumsum()
    print("Total Trades:", len(results_df))
    print("Win Rate:", (results_df['result'] == 'Target').mean())
    print("Total PnL:", results_df['pnl'].sum())
else:
    print("No trades generated — backtest summary skipped.")


print("Total Trades:", len(results_df))
print("Total PnL:", results_df['pnl'].sum())
print(f"Number of trades found: {len(results)}")



C:\Users\Admin\AppData\Local\Temp\ipykernel_2388\3396363271.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('symbol', group_keys=False).apply(add_rsi).reset_index(drop=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2388\3396363271.py:54: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eod_rsi = df.groupby(['symbol', 'date'], group_keys=False).apply(


Number of trades: 0
⚠ No trades generated — conditions likely too strict.
No trades generated — backtest summary skipped.
Total Trades: 0


KeyError: 'pnl'

In [15]:
import pandas as pd
import numpy as np
from datetime import timedelta
import ta  # lightweight technical analysis library (no TA-Lib headaches)

# ----------------------------
# STEP 1: LOAD & PREPARE DATA
# ----------------------------
df = pd.read_csv("all_stocks_ohlc.csv", parse_dates=['timestamp'])
df.columns = [c.lower() for c in df.columns]
df.sort_values(by=['symbol', 'timestamp'], inplace=True)

# Add RSI (14) using 'ta' library
df['rsi'] = df.groupby('symbol', group_keys=False).apply(
    lambda x: ta.momentum.RSIIndicator(x['close'], window=14).rsi()
).reset_index(drop=True)

# Extract date/time columns
df['date'] = df['timestamp'].dt.date
df['time'] = df['timestamp'].dt.time

# ----------------------------
# STEP 2: DAILY AGGREGATIONS
# ----------------------------
daily = df.groupby(['symbol', 'date']).agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum',
}).reset_index()

# Previous day high
daily['prev_day_high'] = daily.groupby('symbol')['high'].shift(1)

# Weekly high (rolling 5 trading days)
daily['weekly_high'] = daily.groupby('symbol')['high'].rolling(window=5, min_periods=1).max().reset_index(level=0, drop=True)

# Average 5-day volume
daily['avg_5_vol'] = daily.groupby('symbol')['volume'].rolling(window=5, min_periods=1).mean().reset_index(level=0, drop=True)

# ----------------------------
# STEP 3: RELAXED SIGNAL LOGIC
# ----------------------------
# Relaxed conditions to ensure trades occur for testing
daily['cond1'] = daily['volume'] > 1.05 * daily['avg_5_vol']  # smaller multiplier
daily['cond2'] = daily['close'] >= daily['prev_day_high']     # allow equal to
daily['cond3'] = daily['close'] >= daily['weekly_high']       # allow equal to
daily['cond4'] = True  # ignore RSI for testing

# Final signal = all relaxed conditions
daily['final_signal'] = daily['cond1'] & daily['cond2'] & daily['cond3'] & daily['cond4']

# ----------------------------
# STEP 4: NEXT-DAY TRADE EXECUTION
# ----------------------------
results = []

for idx, row in daily.iterrows():
    if not row['final_signal']:
        continue

    symbol = row['symbol']
    signal_date = pd.to_datetime(row['date'])
    next_day = signal_date + timedelta(days=1)

    # Get next day's 5-min data
    next_day_data = df[(df['symbol'] == symbol) & (df['timestamp'].dt.date == next_day.date())]
    if next_day_data.empty:
        continue

    # First 15-min candle
    candle = next_day_data[
        (next_day_data['timestamp'].dt.time >= pd.to_datetime("09:15").time()) &
        (next_day_data['timestamp'].dt.time <= pd.to_datetime("09:30").time())
    ]
    if candle.empty:
        continue

    entry = candle['high'].max() + 0.05  # buffer
    sl = candle['low'].min()
    target = entry + 1.5 * (entry - sl)

    # Track intraday after 9:30
    day_data = next_day_data[next_day_data['timestamp'].dt.time >= pd.to_datetime("09:30").time()]
    entry_hit = day_data.iloc[[0]]
    entry_time = entry_hit.iloc[0]['timestamp']
    trade_data = day_data[day_data['timestamp'] >= entry_time]

    # Determine exit
    hit_price = None
    exit_time = None
    result = "Time Exit"

    for _, r in trade_data.iterrows():
        if r['low'] <= sl:
            hit_price = sl
            exit_time = r['timestamp']
            result = "SL"
            break
        elif r['high'] >= target:
            hit_price = target
            exit_time = r['timestamp']
            result = "Target"
            break

    if hit_price is None:
        # Exit at 3:20 PM
        exit_rows = trade_data[trade_data['timestamp'].dt.time <= pd.to_datetime("15:20").time()]
        if exit_rows.empty:
            continue
        exit_row = exit_rows.iloc[-1]
        hit_price = exit_row['close']
        exit_time = exit_row['timestamp']

    pnl = hit_price - entry

    results.append({
        'symbol': symbol,
        'signal_date': signal_date,
        'entry_date': next_day,
        'entry_time': entry_time,
        'entry_price': entry,
        'exit_price': hit_price,
        'exit_time': exit_time,
        'result': result,
        'pnl': pnl
    })

# ----------------------------
# STEP 5: BACKTEST SUMMARY
# ----------------------------
results_df = pd.DataFrame(results)

if not results_df.empty:
    results_df['cumulative_pnl'] = results_df['pnl'].cumsum()
    print("✅ Backtest Complete")
    print("Total Trades:", len(results_df))
    print("Win Rate:", (results_df['result'] == 'Target').mean())
    print("Total PnL:", results_df['pnl'].sum())
    results_df.to_csv("backtest_results.csv", index=False)
else:
    print("⚠ No trades generated — even with relaxed filters. Check data formatting or symbol names.")


C:\Users\Admin\AppData\Local\Temp\ipykernel_2388\1869106972.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['rsi'] = df.groupby('symbol', group_keys=False).apply(


⚠ No trades generated — even with relaxed filters. Check data formatting or symbol names.


In [ ]:
import pandas as pd
import numpy as np
from datetime import time, timedelta
import ta

# --------- CONFIG ----------
CSV_FILE = "all_stocks_ohlc.csv"   # change path if needed
TIMESTAMP_COL = "timestamp"
SYMBOL_COL = "symbol"              # change if your symbol column named differently
# Trading times (5-min candles)
SIGNAL_CANDLE_TIME = time(15, 15)  # signal evaluated at 15:15 candle close
ENTRY_CANDLE_TIME = time(9, 30)    # entry at close of 09:30 candle next day
SQUARE_OFF_TIME = time(15, 20)     # force exit at or before 15:20
RSI_PERIOD = 14
WEEK_LOOKBACK = 5                  # "last week high" ~ last 5 trading days
VOL_LOOKBACK = 5                   # avg over past 5 trading days
RISK_REWARD = 1.5

# --------- LOAD & PREP ----------
df = pd.read_csv(CSV_FILE, parse_dates=[TIMESTAMP_COL])
df.columns = [c.lower() for c in df.columns]

# If symbol column not present try guess
if SYMBOL_COL not in df.columns:
    possible = [c for c in df.columns if c not in ['timestamp','open','high','low','close','volume','openinterest']]
    if possible:
        SYMBOL_COL = possible[0]
    else:
        raise KeyError("Symbol column not found. Please ensure a 'symbol' column exists in CSV.")

# ensure sorted within each symbol
df.sort_values([SYMBOL_COL, TIMESTAMP_COL], inplace=True)
df.reset_index(drop=True, inplace=True)

# add date/time helpers
df['date'] = df[TIMESTAMP_COL].dt.date
df['time'] = df[TIMESTAMP_COL].dt.time

# --------- RSI per symbol (on 5-min series) ----------
def compute_rsi(group):
    rsi = ta.momentum.RSIIndicator(close=group['close'], window=RSI_PERIOD).rsi()
    group = group.assign(rsi=rsi.values)
    return group

df = df.groupby(SYMBOL_COL, group_keys=False).apply(compute_rsi).reset_index(drop=True)

# --------- Prepare daily aggregates (per symbol) ----------
# daily total volume, daily high, etc.
daily = df.groupby([SYMBOL_COL, 'date']).agg(
    day_open = ('open','first'),
    day_high = ('high','max'),
    day_low = ('low','min'),
    day_close = ('close','last'),
    day_volume = ('volume','sum')
).reset_index()

# previous day high and rolling weekly high (last 5 trading days)
daily['prev_day_high'] = daily.groupby(SYMBOL_COL)['day_high'].shift(1)
daily['weekly_high'] = daily.groupby(SYMBOL_COL)['day_high'].rolling(window=WEEK_LOOKBACK, min_periods=1).max().reset_index(level=0, drop=True)
daily['avg_5day_vol'] = daily.groupby(SYMBOL_COL)['day_volume'].rolling(window=VOL_LOOKBACK, min_periods=1).mean().reset_index(level=0, drop=True)

# --------- Build mapping for quick intraday lookup ----------
# We'll need to quickly get:
# - the 15:15 candle row for a given symbol & date
# - the next day's 09:30 candle close (entry)
# - intraday series for next day after entry
# Build group dicts for faster access
symbol_groups = {sym: g.reset_index(drop=True) for sym, g in df.groupby(SYMBOL_COL)}

# --------- SIGNAL GENERATION (based on 15:15 candle) ----------
signals = []  # list of dicts with symbol and signal_date

for idx, dr in daily.iterrows():
    sym = dr[SYMBOL_COL]
    cur_date = dr['date']

    # locate that symbol's intraday data
    intraday = symbol_groups.get(sym)
    if intraday is None:
        continue

    # get all rows for that date
    day_rows = intraday[intraday['date'] == cur_date]
    if day_rows.empty:
        continue

    # total volume today already in daily['day_volume']
    today_total_vol = dr['day_volume']
    avg_5_vol = dr['avg_5day_vol'] if not pd.isna(dr['avg_5day_vol']) else np.nan

    # Volume condition: today_total_vol > 1.0 * avg_5_vol
    cond_vol = False
    if not np.isnan(avg_5_vol):
        cond_vol = (today_total_vol > 1.0 * avg_5_vol)

    # get 15:15 candle row
    candle_1515 = day_rows[day_rows['time'] == SIGNAL_CANDLE_TIME]
    if candle_1515.empty:
        # no 15:15 candle — skip
        continue
    candle_1515 = candle_1515.iloc[-1]  # should be single row

    # Price conditions
    cond_prev_high = False
    if not pd.isna(dr['prev_day_high']):
        cond_prev_high = (candle_1515['close'] > dr['prev_day_high'])

    cond_week_high = False
    if not pd.isna(dr['weekly_high']):
        cond_week_high = (candle_1515['close'] > dr['weekly_high'])

    # RSI condition: check cross above 70 at 15:15
    # Need previous rsi value in same day (or the last available prior index)
    # We'll find previous row in intraday before this timestamp
    idx_c = candle_1515.name  # original index in the intraday group
    # but since we used .iloc above, we don't have direct positional index; instead compute via timestamp
    ts = candle_1515[TIMESTAMP_COL]
    prev_rows = day_rows[day_rows[TIMESTAMP_COL] < ts]
    cond_rsi = False
    if 'rsi' in candle_1515.index:
        rsi_now = candle_1515['rsi']
        rsi_prev = prev_rows['rsi'].iloc[-1] if not prev_rows.empty else np.nan
        # require prev RSI <= 70 and current RSI > 70 (cross above)
        if (not np.isnan(rsi_now)) and (not np.isnan(rsi_prev)):
            cond_rsi = (rsi_prev <= 70) and (rsi_now > 70)
        else:
            # fallback: require rsi_now > 70
            cond_rsi = (not np.isnan(rsi_now)) and (rsi_now > 70)

    # final signal: all conditions must be True
    if cond_vol and cond_prev_high and cond_week_high and cond_rsi:
        signals.append({
            'symbol': sym,
            'signal_date': cur_date,
            'signal_timestamp': candle_1515[TIMESTAMP_COL],
            'candle_close_1515': candle_1515['close'],
            'today_total_vol': today_total_vol,
            'avg_5day_vol': avg_5_vol,
            'rsi_prev': float(rsi_prev) if 'rsi_prev' in locals() else None,
            'rsi_now': float(rsi_now) if 'rsi_now' in locals() else None
        })

# Debug: how many signals found
print(f"Signals found: {len(signals)}")

# --------- EXECUTE NEXT-DAY TRADES ----------
trades = []
for s in signals:
    sym = s['symbol']
    sig_date = pd.to_datetime(s['signal_date'])
    entry_date = (sig_date + timedelta(days=1)).date()

    intraday = symbol_groups.get(sym)
    if intraday is None:
        continue

    next_day_rows = intraday[intraday['date'] == entry_date]
    if next_day_rows.empty:
        # no next day data (holiday or missing) — skip
        continue

    # find the 09:30 candle (entry candle) for next day
    entry_candle = next_day_rows[next_day_rows['time'] == ENTRY_CANDLE_TIME]
    if entry_candle.empty:
        # no 09:30 candle — fallback to first available candle after 09:30
        entry_candle = next_day_rows[next_day_rows['time'] > ENTRY_CANDLE_TIME]
        if entry_candle.empty:
            # fallback to first candle of the day
            entry_candle = next_day_rows.iloc[[0]]
    entry_candle = entry_candle.iloc[0]

    entry_price = float(entry_candle['close'])
    sl = float(entry_candle['low'])
    target = entry_price + RISK_REWARD * (entry_price - sl)

    # Build the trade_data: all intraday rows for that next day after entry_time (inclusive)
    trade_after = next_day_rows[next_day_rows[TIMESTAMP_COL] >= entry_candle[TIMESTAMP_COL]].copy()
    if trade_after.empty:
        continue

    exit_price = None
    exit_time = None
    exit_type = "Time Exit"

    # iterate rows to see if SL or Target hit
    for _, r in trade_after.iterrows():
        if r['low'] <= sl:
            exit_price = sl
            exit_time = r[TIMESTAMP_COL]
            exit_type = "SL"
            break
        if r['high'] >= target:
            exit_price = target
            exit_time = r[TIMESTAMP_COL]
            exit_type = "Target"
            break

    # If none hit, force exit at last available bar <= SQUARE_OFF_TIME
    if exit_price is None:
        candidates = trade_after[trade_after['time'] <= SQUARE_OFF_TIME]
        if not candidates.empty:
            last_row = candidates.iloc[-1]
        else:
            # fallback to last available row of that day
            last_row = trade_after.iloc[-1]
        exit_price = float(last_row['close'])
        exit_time = last_row[TIMESTAMP_COL]
        exit_type = "Time Exit"

    pnl = exit_price - entry_price

    trades.append({
        'symbol': sym,
        'signal_date': s['signal_date'],
        'entry_date': entry_date,
        'entry_time': entry_candle[TIMESTAMP_COL],
        'entry_price': entry_price,
        'exit_time': exit_time,
        'exit_price': exit_price,
        'exit_type': exit_type,
        'pnl': pnl
    })

# --------- SUMMARY ----------
trades_df = pd.DataFrame(trades)
if trades_df.empty:
    print("No trades executed (signals may be zero or next-day data missing).")
else:
    trades_df['cumulative_pnl'] = trades_df['pnl'].cumsum()
    wins = trades_df['exit_type'] == 'Target'
    print("Total trades:", len(trades_df))
    print("Wins (target hit):", wins.sum())
    print("Win rate:", wins.mean())
    print("Total PnL:", trades_df['pnl'].sum())
    print("\nSample trades:")
    print(trades_df.head().to_string(index=False))

    # save results
    trades_df.to_csv("backtest_trades_exact_1515_to_0930.csv", index=False)
    print("Saved trades to backtest_trades_exact_1515_to_0930.csv")


C:\Users\Admin\AppData\Local\Temp\ipykernel_2388\589687931.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(SYMBOL_COL, group_keys=False).apply(compute_rsi).reset_index(drop=True)


Signals found: 0
No trades executed (signals may be zero or next-day data missing).


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load data
file = 'all_stocks_ohlc.csv'
df = pd.read_csv(file)

# Convert timestamp and prepare data
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date
df['time'] = df['timestamp'].dt.time
df = df.sort_values(['symbol', 'timestamp']).reset_index(drop=True)

print("🚀 COMPLETE Canon Momentum Strategy - ENTIRE CSV Analysis")
print("=" * 70)

# Get ALL unique trading dates from CSV
unique_dates = sorted(df['date'].unique())
total_dates = len(unique_dates)

print(f"📊 COMPLETE DATASET ANALYSIS:")
print(f"Total rows in CSV: {len(df):,}")
print(f"Total stocks: {df['symbol'].nunique()}")
print(f"Total trading dates: {total_dates}")
print(f"Date range: {unique_dates[0]} to {unique_dates[-1]}")
print(f"Period: {(pd.to_datetime(unique_dates[-1]) - pd.to_datetime(unique_dates[0])).days + 1} calendar days")

# Verify we have enough data
if total_dates < 6:
    print("❌ Need at least 6 days of data (5 for analysis + 1 for trading)")
    exit()

# Split into analysis and trading periods
analysis_days = unique_dates[:5]  # First 5 days for volume analysis
trading_days = unique_dates[5:]   # ALL remaining days for trading

print(f"\n📈 ANALYSIS PERIOD (No Trading): {analysis_days[0]} to {analysis_days[-1]} ({len(analysis_days)} days)")
print(f"🎯 TRADING PERIOD: {trading_days[0]} to {trading_days[-1]} ({len(trading_days)} days)")

# Build 5-day volume baseline for ALL stocks
print(f"\n📊 Building Volume History for ALL {df['symbol'].nunique()} Stocks...")

daily_volumes = {}
stocks_with_data = []

for symbol in df['symbol'].unique():
    symbol_data = df[df['symbol'] == symbol]
    
    # Calculate daily volumes for analysis period
    daily_vol = []
    for date in analysis_days:
        day_data = symbol_data[symbol_data['date'] == date]
        if len(day_data) > 0:
            total_volume = day_data['volume'].sum()
            daily_vol.append(total_volume)
        else:
            daily_vol.append(0)
    
    # Only include stocks with data in all 5 analysis days
    if sum(daily_vol) > 0 and len([v for v in daily_vol if v > 0]) >= 3:  # At least 3 days of data
        daily_volumes[symbol] = daily_vol
        stocks_with_data.append(symbol)

print(f"✅ Volume baseline calculated for {len(stocks_with_data)} stocks")
print(f"Stocks ready for trading: {stocks_with_data[:10]}{'...' if len(stocks_with_data) > 10 else ''}")

# Momentum calculation function
def calculate_momentum_signals(df_symbol, analysis_volumes):
    """Calculate 4 momentum indicators for a symbol"""
    df_symbol = df_symbol.copy()
    
    # RSI calculation (14 period)
    def calculate_rsi(prices, period=14):
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi
    
    df_symbol['rsi'] = calculate_rsi(df_symbol['close'])
    df_symbol['rsi_signal'] = df_symbol['rsi'] > 70
    
    # Daily aggregations
    daily_data = df_symbol.groupby('date').agg({
        'high': 'max',
        'close': 'last',
        'volume': 'sum'
    })
    
    # Volume signal: today > 1.5x average of analysis period
    avg_volume_5d = np.mean(analysis_volumes) if len(analysis_volumes) > 0 else 0
    daily_data['volume_signal'] = daily_data['volume'] > (1 * avg_volume_5d)
    
    # Price signals
    daily_data['prev_high'] = daily_data['high'].shift(1)
    daily_data['close_above_prev_high'] = daily_data['close'] > daily_data['prev_high']
    
    daily_data['week_high_5d'] = daily_data['high'].rolling(window=5).max().shift(1)
    daily_data['close_above_week_high'] = daily_data['close'] > daily_data['week_high_5d']
    
    # Map back to intraday data
    date_mappings = {
        'volume_signal': daily_data['volume_signal'].to_dict(),
        'close_above_prev_high': daily_data['close_above_prev_high'].to_dict(),
        'close_above_week_high': daily_data['close_above_week_high'].to_dict()
    }
    
    for signal, mapping in date_mappings.items():
        df_symbol[signal] = df_symbol['date'].map(mapping)
    
    # Count signals (need 3+ out of 4)
    df_symbol['signal_count'] = (
        df_symbol['volume_signal'].fillna(False).astype(int) +
        df_symbol['rsi_signal'].fillna(False).astype(int) +
        df_symbol['close_above_prev_high'].fillna(False).astype(int) +
        df_symbol['close_above_week_high'].fillna(False).astype(int)
    )
    
    df_symbol['bullish_momentum'] = df_symbol['signal_count'] >= 3
    
    return df_symbol

# ANALYZE ALL TRADING DAYS
print(f"\n🎯 ANALYZING ALL {len(trading_days)} TRADING DAYS...")
print("=" * 70)

all_trading_results = []
daily_summary = []

for day_num, target_date in enumerate(trading_days, 1):
    
    # Show progress every 5 days
    if day_num % 5 == 0 or day_num == len(trading_days):
        print(f"📅 Processing Day {day_num}/{len(trading_days)}: {target_date}")
    
    # Get data for this trading day
    day_data = df[df['date'] == target_date].copy()
    
    if len(day_data) == 0:
        continue
    
    # Find momentum stocks for this day
    momentum_stocks = []
    
    for symbol in stocks_with_data:
        if symbol not in day_data['symbol'].values:
            continue
            
        # Get historical data up to previous day
        prev_dates = unique_dates[:unique_dates.index(target_date)]
        symbol_hist_data = df[
            (df['symbol'] == symbol) & 
            (df['date'].isin(prev_dates))
        ].copy()
        
        if len(symbol_hist_data) < 10:  # Need minimum data
            continue
            
        # Calculate momentum signals
        try:
            symbol_with_signals = calculate_momentum_signals(symbol_hist_data, daily_volumes[symbol])
            
            # Check if previous day showed momentum
            if len(prev_dates) > 0:
                last_day_data = symbol_with_signals[symbol_with_signals['date'] == prev_dates[-1]]
                if len(last_day_data) > 0 and last_day_data['bullish_momentum'].any():
                    momentum_stocks.append(symbol)
        except:
            continue
    
    # Execute trades for momentum stocks
    day_trades = 0
    day_setups = 0
    day_pnl = 0
    
    for stock in momentum_stocks:
        stock_day_data = day_data[day_data['symbol'] == stock].copy()
        if len(stock_day_data) == 0:
            continue
            
        day_setups += 1
        
        # Trading setup
        first_candle = stock_day_data.iloc[0]
        entry_price = first_candle['high'] + 0.10
        stop_loss = entry_price * 0.98  # 2% stop loss
        target = entry_price + ((entry_price - stop_loss) * 1.5)  # 1:1.5 ratio
        
        # Check execution
        day_high = stock_day_data['high'].max()
        day_low = stock_day_data['low'].min()
        
        if day_high >= entry_price:  # Trade triggered
            day_trades += 1
            
            if day_high >= target:
                outcome = "TARGET"
                pnl = target - entry_price
            elif day_low <= stop_loss:
                outcome = "STOP_LOSS"
                pnl = stop_loss - entry_price
            else:
                outcome = "SQUARE_OFF"
                pnl = stock_day_data.iloc[-1]['close'] - entry_price
        else:
            outcome = "NO_ENTRY"
            pnl = 0
        
        pnl_pct = (pnl / entry_price) * 100 if entry_price > 0 else 0
        day_pnl += pnl_pct
        
        all_trading_results.append({
            'day_num': day_num,
            'date': target_date,
            'symbol': stock,
            'entry': entry_price,
            'target': target,
            'stop_loss': stop_loss,
            'outcome': outcome,
            'pnl_pct': pnl_pct
        })
    
    # Daily summary
    daily_summary.append({
        'day_num': day_num,
        'date': target_date,
        'momentum_stocks': len(momentum_stocks),
        'setups': day_setups,
        'trades': day_trades,
        'day_pnl': day_pnl
    })

# FINAL PERFORMANCE SUMMARY
print(f"\n🏆 COMPLETE STRATEGY PERFORMANCE SUMMARY")
print("=" * 70)

if all_trading_results:
    results_df = pd.DataFrame(all_trading_results)
    daily_df = pd.DataFrame(daily_summary)
    
    # Overall statistics
    total_setups = len(results_df)
    total_trades = len(results_df[results_df['outcome'] != 'NO_ENTRY'])
    winning_trades = len(results_df[results_df['pnl_pct'] > 0])
    losing_trades = len(results_df[results_df['pnl_pct'] < 0])
    
    print(f"📊 COMPLETE ANALYSIS RESULTS:")
    print(f"Analysis Period: {analysis_days[0]} to {analysis_days[-1]} ({len(analysis_days)} days)")
    print(f"Trading Period: {trading_days[0]} to {trading_days[-1]} ({len(trading_days)} days)")
    print(f"Total Trading Days Analyzed: {len(trading_days)}")
    
    print(f"\n📈 TRADING STATISTICS:")
    print(f"Total Momentum Setups: {total_setups}")
    print(f"Actual Trades Executed: {total_trades}")
    print(f"Winning Trades: {winning_trades}")
    print(f"Losing Trades: {losing_trades}")
    
    if total_trades > 0:
        win_rate = (winning_trades / total_trades) * 100
        avg_profit = results_df[results_df['pnl_pct'] > 0]['pnl_pct'].mean() if winning_trades > 0 else 0
        avg_loss = results_df[results_df['pnl_pct'] < 0]['pnl_pct'].mean() if losing_trades > 0 else 0
        total_pnl = results_df['pnl_pct'].sum()
        
        print(f"Win Rate: {win_rate:.1f}%")
        print(f"Average Profit: {avg_profit:.2f}%")
        print(f"Average Loss: {avg_loss:.2f}%")
        print(f"Total P&L: {total_pnl:+.2f}%")
        print(f"Average P&L per Trade: {total_pnl/total_trades:.2f}%")
        
        # Best performing stocks
        stock_performance = results_df.groupby('symbol')['pnl_pct'].agg(['count', 'sum', 'mean']).round(2)
        stock_performance.columns = ['Trades', 'Total_PnL', 'Avg_PnL']
        stock_performance = stock_performance.sort_values('Total_PnL', ascending=False)
        
        print(f"\n🏆 TOP PERFORMING STOCKS:")
        print(stock_performance.head(10))
        
        # Weekly performance
        results_df['week'] = pd.to_datetime(results_df['date']).dt.isocalendar().week
        weekly_pnl = results_df.groupby('week')['pnl_pct'].sum().round(2)
        
        print(f"\n📅 WEEKLY P&L:")
        for week, pnl in weekly_pnl.items():
            week_dates = results_df[results_df['week'] == week]['date'].unique()
            print(f"Week {week} ({week_dates[0]} to {week_dates[-1]}): {pnl:+.2f}%")

# SAVE ALL ANALYSIS TO CSV FILES
print(f"\n💾 SAVING ANALYSIS TO CSV FILES...")

if all_trading_results:
    # 1. Complete trading results
    results_df = pd.DataFrame(all_trading_results)
    results_df.to_csv('momentum_strategy_complete_results.csv', index=False)
    print(f"✅ Saved: momentum_strategy_complete_results.csv ({len(results_df)} rows)")
    
    # 2. Daily summary
    daily_df = pd.DataFrame(daily_summary)
    daily_df.to_csv('momentum_strategy_daily_summary.csv', index=False)
    print(f"✅ Saved: momentum_strategy_daily_summary.csv ({len(daily_df)} rows)")
    
    # 3. Stock performance summary
    stock_performance = results_df.groupby('symbol').agg({
        'pnl_pct': ['count', 'sum', 'mean', 'std'],
        'outcome': lambda x: (x == 'TARGET').sum(),  # Target hits
        'entry': 'mean'  # Average entry price
    }).round(3)
    
    stock_performance.columns = ['Total_Trades', 'Total_PnL_%', 'Avg_PnL_%', 'PnL_StdDev', 'Target_Hits', 'Avg_Entry_Price']
    stock_performance['Win_Rate_%'] = (stock_performance['Target_Hits'] / stock_performance['Total_Trades'] * 100).round(1)
    stock_performance = stock_performance.sort_values('Total_PnL_%', ascending=False)
    stock_performance.to_csv('momentum_strategy_stock_performance.csv')
    print(f"✅ Saved: momentum_strategy_stock_performance.csv ({len(stock_performance)} stocks)")

    # 4. Volume baseline data used for analysis
 volume_baseline_df = pd.DataFrame([
 {'symbol': symbol, 'day1_volume': vols[0], 'day2_volume': vols[1], 
 'day3_volume': vols[2], 'day4_volume': vols[3], 'day5_volume': vols[4],
 'avg_volume_5day': np.mean(vols), 'threshold_1.5x': np.mean(vols) * 1.5}
 for symbol, vols in daily_volumes.items()
 ])
 volume_baseline_df.to_csv('momentum_strategy_volume_baseline.csv', index=False)
 print(f"✅ Saved: momentum_strategy_volume_baseline.csv ({len(volume_baseline_df)} stocks)")
 
 # 5. Detailed signals for each trading day (sample of first 100 records)
 if len(results_df) > 0:
 detailed_signals = []
 for _, trade in results_df.head(100).iterrows(): # Save sample for memory efficiency
 detailed_signals.append({
 'date': trade['date'],
 'symbol': trade['symbol'],
 'day_num': trade['day_num'],
 'entry_price': trade['entry'],
 'stop_loss': trade['stop_loss'], 
 'target': trade['target'],
 'outcome': trade['outcome'],
 'pnl_percent': trade['pnl_pct'],
 'risk_reward_ratio': '1:1.5'
 })
 
 detailed_df = pd.DataFrame(detailed_signals)
 detailed_df.to_csv('momentum_strategy_detailed_trades.csv', index=False)
 print(f"✅ Saved: momentum_strategy_detailed_trades.csv (sample of {len(detailed_df)} trades)")

# 6. Analysis period summary
analysis_summary = {
 'analysis_start_date': analysis_days[0],
 'analysis_end_date': analysis_days[-1],
 'trading_start_date': trading_days[0],
 'trading_end_date': trading_days[-1],
 'total_stocks_analyzed': len(stocks_with_data),
 'total_trading_days': len(trading_days),
 'total_rows_processed': len(df),
 'strategy_description': 'Canon Momentum Strategy with 5-day volume baseline'
}

pd.DataFrame([analysis_summary]).to_csv('momentum_strategy_analysis_summary.csv', index=False)
print(f"✅ Saved: momentum_strategy_analysis_summary.csv")

print(f"\n📁 ALL CSV FILES CREATED:")
print("1. momentum_strategy_complete_results.csv - All individual trades")
print("2. momentum_strategy_daily_summary.csv - Daily performance") 
print("3. momentum_strategy_stock_performance.csv - Stock-wise results")
print("4. momentum_strategy_volume_baseline.csv - 5-day volume data")
print("5. momentum_strategy_detailed_trades.csv - Sample detailed trades")
print("6. momentum_strategy_analysis_summary.csv - Analysis overview")

print(f"\n✅ COMPLETE CSV ANALYSIS FINISHED!")
print(f"🎯 Analyzed {len(df):,} rows across {len(trading_days)} trading days")
print(f"💾 All results saved to CSV files in current directory")

🚀 COMPLETE Canon Momentum Strategy - ENTIRE CSV Analysis
📊 COMPLETE DATASET ANALYSIS:
Total rows in CSV: 57,375
Total stocks: 51
Total trading dates: 45
Date range: 2025-05-26 to 2025-07-25
Period: 61 calendar days

📈 ANALYSIS PERIOD (No Trading): 2025-05-26 to 2025-05-30 (5 days)
🎯 TRADING PERIOD: 2025-06-02 to 2025-07-25 (40 days)

📊 Building Volume History for ALL 51 Stocks...
✅ Volume baseline calculated for 51 stocks
Stocks ready for trading: ['ADANIENT', 'ADANIPORTS', 'APOLLOHOSP', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BEL', 'BHARTIARTL']...

🎯 ANALYZING ALL 40 TRADING DAYS...
📅 Processing Day 5/40: 2025-06-06
📅 Processing Day 10/40: 2025-06-13
📅 Processing Day 15/40: 2025-06-20
📅 Processing Day 20/40: 2025-06-27
📅 Processing Day 25/40: 2025-07-04
📅 Processing Day 30/40: 2025-07-11
📅 Processing Day 35/40: 2025-07-18
📅 Processing Day 40/40: 2025-07-25

🏆 COMPLETE STRATEGY PERFORMANCE SUMMARY
📊 COMPLETE ANALYSIS RESULTS:
Analysis Period: 2025-05-26 to